In [8]:
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

In [9]:
class LeNet:
    @staticmethod
    def build(width, height, depth, classes):
        model = Sequential()
        inputShape = (height, width, depth)
        
        model.add(Conv2D(20, (5, 5), padding="same", input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
        
        model.add(Conv2D(50, (5, 5), padding="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation("relu"))
        
        
        


In [10]:
print("loading images")
data = []
labels = []

imagePaths = sorted(list(paths.list_images("augData/trialDataset")))
random.seed(42)
random.shuffle(imagePaths)

for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (32, 32)).flatten()
    data.append(image)
    
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

loading images


KeyboardInterrupt: 

In [ ]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [ ]:
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.25, random_state=42)

lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)


In [ ]:
model = Sequential()
model.add(Dense(1024, input_shape=(3072,), activation="relu"))
model.add(Dense(512, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

2022-04-21 13:20:59.613837: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/cv2/../../lib64:
2022-04-21 13:20:59.613891: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-21 13:20:59.613961: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (awi-stavya): /proc/driver/nvidia/version does not exist
2022-04-21 13:20:59.614647: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
INIT_LR = 0.01
EPOCHS = 15

print("training network")
opt = SGD(lr=INIT_LR)

model.compile(loss="categorical_crossentropy", optimizer=opt,
            metrics=["accuracy"])

H = model.fit(x=trainX, y=trainY, validation_data=(testX, testY), epochs=EPOCHS, batch_size=15)

training network


/home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/15


ValueError: in user code:

    File "/home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/keras/engine/training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/keras/engine/training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "/home/awiros-tech

In [7]:
print("[INFO] evaluating network...")
predictions = model.predict(x=testX, batch_size=32)

print(predictions)

[INFO] evaluating network...
[[0.10755491]
 [0.02184999]
 [0.01000229]
 [0.00557685]
 [0.00634855]
 [0.0157766 ]
 [0.23415616]
 [0.02429789]
 [0.02505103]
 [0.00745329]
 [0.25773048]
 [0.11697316]
 [0.01051015]
 [0.00941813]
 [0.01263076]
 [0.00830886]
 [0.20066965]
 [0.01239496]
 [0.01450333]
 [0.11489454]
 [0.06123903]
 [0.05483478]
 [0.00816372]
 [0.01203501]
 [0.01326588]
 [0.01220661]
 [0.01245239]
 [0.01550412]
 [0.01239246]
 [0.0281291 ]
 [0.15327346]
 [0.0071958 ]
 [0.01824808]
 [0.00515196]
 [0.24493855]
 [0.29627076]
 [0.01612854]
 [0.00643656]
 [0.01475823]
 [0.01370621]
 [0.00484252]
 [0.01578775]
 [0.17160362]
 [0.0294123 ]
 [0.00413272]
 [0.19780329]
 [0.41267422]
 [0.03416651]
 [0.00843439]
 [0.01063851]
 [0.19616967]
 [0.01138291]
 [0.00867218]
 [0.01014963]
 [0.1338084 ]
 [0.01840985]
 [0.0196867 ]
 [0.00821453]
 [0.00647968]
 [0.30290812]
 [0.01167735]
 [0.10808352]
 [0.03125101]
 [0.01547852]
 [0.01322669]
 [0.00494051]
 [0.01171413]
 [0.05785522]
 [0.0162403 ]
 [0.0

In [ ]:
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))